In [10]:
#import and load data file
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intent.json').read()
intents = json.loads(data_file)

In [16]:
#preproccessdata
def pullData(d1,d2):
    df = pd.DataFrame(columns=[d1,d2])

    for intent in intents['intents']:
        
        for pattern in intent['text']:
            

        #tokenize each word
            w = nltk.word_tokenize(pattern)
            words.extend(w)
            df_to_append = pd.Series([w,intent['intent']], index = df.columns)
            df = df.append(df_to_append,ignore_index=True)
        #add documents in the corpus
            documents.append((w, intent['intent']))
            for response in intent['responses']:
                
                w = response
                df_to_append = pd.Series([w,intent['intent']], index = df.columns)
                df = df.append(df_to_append,ignore_index=True)
        
                if intent['intent'] not in classes:
                   classes.append(intent['intent'])
    return df
    


In [17]:
data1=pullData('questions','labels')
data1.head(15)

,questions,labels
0,[Hi],Greeting
1,"Hi human, please tell me your user",Greeting
2,"Hello human, please tell me your user",Greeting
3,"Hola human, please tell me your user",Greeting
4,"[Hi, there]",Greeting
5,"Hi human, please tell me your user",Greeting
6,"Hello human, please tell me your user",Greeting
7,"Hola human, please tell me your user",Greeting
8,[Hola],Greeting
9,"Hi human, please tell me your user",Greeting


In [18]:
data2=pullData('response','labels')
data2.head(15)

,response,labels
0,[Hi],Greeting
1,"Hi human, please tell me your user",Greeting
2,"Hello human, please tell me your user",Greeting
3,"Hola human, please tell me your user",Greeting
4,"[Hi, there]",Greeting
5,"Hi human, please tell me your user",Greeting
6,"Hello human, please tell me your user",Greeting
7,"Hola human, please tell me your user",Greeting
8,[Hola],Greeting
9,"Hi human, please tell me your user",Greeting


In [19]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

715 documents
22 classes ['Clever', 'CourtesyGoodBye', 'CourtesyGreeting', 'CourtesyGreetingResponse', 'CurrentHumanQuery', 'GoodBye', 'Gossip', 'Greeting', 'GreetingResponse', 'Jokes', 'NameQuery', 'NotTalking2U', 'PodBayDoor', 'PodBayDoorResponse', 'RealNameQuery', 'SelfAware', 'Shutup', 'Swearing', 'Thanks', 'TimeQuery', 'UnderstandQuery', 'WhoAmI']
117 unique lemmatized words ["'s", ',', 'a', 'about', 'adam', 'adios', 'am', 'any', 'anyone', 'are', 'aware', 'bay', 'be', 'bella', 'bored', 'by', 'bye', 'call', 'camera', 'can', 'cheering', 'clever', 'communicating', 'comprendo', 'conscious', 'could', 'do', 'doing', 'door', 'enough', 'for', 'friend', 'fuck', 'genious', 'get', 'girl', 'give', 'good', 'goodbye', 'gossip', 'got', 'great', 'have', 'hear', 'hello', 'help', 'helpful', 'hi', 'hola', 'hope', 'how', 'hya', 'i', 'identify', 'in', 'intelligent', 'is', 'it', 'joke', 'know', 'later', 'laugh', 'make', 'me', 'mean', 'meant', 'more', 'my', "n't", 'name', 'need', 'not', 'off', 'ok', 'op

In [47]:
test_list = list(data1.groupby(by='labels',as_index=False).first()['questions'])
test_list

[['You', 'are', 'very', 'clever'],
 ['Thanks', ',', 'bye'],
 ['How', 'are', 'you', '?'],
 ['Good', 'thanks', '!', 'My', 'user', 'is', 'Adam'],
 ['What', 'is', 'my', 'name', '?'],
 ['Bye'],
 ['I', 'am', 'bored', 'gossip', 'with', 'me'],
 ['Hi'],
 ['My', 'user', 'is', 'Adam'],
 ['Tell', 'me', 'a', 'joke'],
 ['What', 'is', 'your', 'name', '?'],
 ['I', 'am', 'not', 'talking', 'to', 'you'],
 ['Open', 'the', 'pod', 'bay', 'door'],
 ['Why'],
 ['What', 'is', 'your', 'real', 'name', '?'],
 ['Can', 'you', 'prove', 'you', 'are', 'self-aware'],
 ['Be', 'quiet'],
 ['fuck', 'off'],
 ['OK', 'thank', 'you'],
 ['What', 'is', 'the', 'time', '?'],
 ['Do', 'you', 'understand', 'what', 'I', 'am', 'saying'],
 ['Can', 'you', 'see', 'me', '?']]

In [33]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-33-451aefdaa425>:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [35]:
print("a",output_row)


a [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]


In [15]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
30/30 [==============================] - 1s 3ms/step - loss: 3.1329 - accuracy: 0.0213
Epoch 2/200
30/30 [==============================] - 0s 1ms/step - loss: 3.0535 - accuracy: 0.0647
Epoch 3/200
30/30 [==============================] - 0s 1ms/step - loss: 2.9256 - accuracy: 0.1885
Epoch 4/200
30/30 [==============================] - 0s 1ms/step - loss: 2.7689 - accuracy: 0.2527
Epoch 5/200
30/30 [==============================] - 0s 1ms/step - loss: 2.5459 - accuracy: 0.3248
Epoch 6/200
30/30 [==============================] - 0s 1ms/step - loss: 2.3434 - accuracy: 0.3476
Epoch 7/200
30/30 [==============================] - 0s 1ms/step - loss: 2.2338 - accuracy: 0.4720
Epoch 8/200
30/30 [==============================] - 0s 1ms/step - loss: 1.9147 - accuracy: 0.4157
Epoch 9/200
30/30 [==============================] - 0s 1ms/step - loss: 1.8336 - accuracy: 0.4450
Epoch 10/200
30/30 [==============================] - 0s 1ms/step - loss: 1.5147 - accuracy: 0.6422
Epoch 11/

30/30 [==============================] - 0s 1ms/step - loss: 0.0478 - accuracy: 0.9940
Epoch 163/200
30/30 [==============================] - 0s 1ms/step - loss: 0.0463 - accuracy: 0.9942
Epoch 164/200
30/30 [==============================] - 0s 1ms/step - loss: 0.0753 - accuracy: 0.9756
Epoch 165/200
30/30 [==============================] - 0s 1ms/step - loss: 0.1616 - accuracy: 0.9489
Epoch 166/200
30/30 [==============================] - 0s 1ms/step - loss: 0.0506 - accuracy: 0.9919
Epoch 167/200
30/30 [==============================] - 0s 1ms/step - loss: 0.1037 - accuracy: 0.9754
Epoch 168/200
30/30 [==============================] - 0s 1ms/step - loss: 0.1114 - accuracy: 0.9535
Epoch 169/200
30/30 [==============================] - 0s 1ms/step - loss: 0.0891 - accuracy: 0.9559
Epoch 170/200
30/30 [==============================] - 0s 1ms/step - loss: 0.0274 - accuracy: 0.9957
Epoch 171/200
30/30 [==============================] - 0s 1000us/step - loss: 0.2219 - accuracy: 0.9693
E